In [6]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append('/Users/aeshef/Documents/GitHub/kursach/pys/data_collection')

import bonds_parser
importlib.reload(bonds_parser)
from bonds_parser import MOEXBondHistoricalParser, Logger


In [8]:
like_print_log = Logger(name="main", format="%(message)s", store=True)

bond_parser = MOEXBondHistoricalParser(log=like_print_log, verbose=False)

result = bond_parser.parse_from_2024_to_now(
    include_non_trading_days=True,  # Включаем выходные дни
    analyze_continuity=True,        # Анализируем непрерывность и создаем полный датасет
    yield_range=(0, 100),           # Широкий диапазон - берем все
    price_range=(0, 100),           # Широкий диапазон - берем все
    duration_range=(0, 7200)         # Широкий диапазон - берем все
)

print(f"Всего дней с данными: {len(result)}")
trading_days = sum(1 for date in result.keys() if datetime.strptime(date, '%Y-%m-%d').weekday() < 5)
print(f"Торговых дней: {trading_days}")
print(f"Нерабочих дней с данными: {len(result) - trading_days}")



Запуск массового парсинга данных с 2024-01-01 по 2025-04-22
Запуск массового парсинга данных с 2024-01-01 по 2025-04-22
Запрошен период с 2024-01-01 по 2025-04-22
Запрошен период с 2024-01-01 по 2025-04-22
Получение облигаций за период с 2024-01-01 по 2025-04-22
Получение облигаций за период с 2024-01-01 по 2025-04-22
Обработка даты: 2024-01-01 (рабочий день)
Обработка даты: 2024-01-01 (рабочий день)
Получение списка облигаций на 2024-01-01
Получение списка облигаций на 2024-01-01
Результаты парсинга облигаций на 2024-01-01:
Результаты парсинга облигаций на 2024-01-01:
  Группа 58: 0 облигаций
  Группа 58: 0 облигаций
  Группа 193: 0 облигаций
  Группа 193: 0 облигаций
  Группа 105: 0 облигаций
  Группа 105: 0 облигаций
  Группа 77: 0 облигаций
  Группа 77: 0 облигаций
  Группа 207: 0 облигаций
  Группа 207: 0 облигаций
  Группа 167: 0 облигаций
  Группа 167: 0 облигаций
  Группа 245: 0 облигаций
  Группа 245: 0 облигаций
  Всего: 0 облигаций
  Всего: 0 облигаций
Нет облигаций для прим

In [ ]:
optimal_threshold = bond_parser.find_optimal_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    min_bonds=20,
    max_threshold=99
)

print(f"Оптимальный порог непрерывности: {optimal_threshold}%")

dataset = bond_parser.generate_dataset_with_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    continuity_threshold=optimal_threshold
)

if dataset is not None:
    print(f"Создан датасет с {len(dataset['secid'].unique())} облигациями и {len(dataset['date'].unique())} датами")
else:
    print("Не удалось создать датасет")

TypeError: an integer is required (got type str)